In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import chi2_contingency
from scipy.stats import chi2
import statsmodels.api as sm
from statsmodels.formula.api import ols
from statsmodels.stats.multicomp import pairwise_tukeyhsd
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, classification_report, roc_auc_score, accuracy_score
from sklearn.metrics import roc_curve
from imblearn.over_sampling import SMOTE
from scipy import stats
import datetime

In [ ]:
# In ra giá trị duy nhất có trong mỗi cột
def giatriduynhat(df, lst_tencot):
    a = 1
    for i in lst_tencot:
        print(a,'/', i,':',df.dtypes[i],':', df[i].unique())
        print('-'*10, "\n")
        a = a + 1

In [ ]:
# Xác định biến liên tục và biến phân loại
def xacdinhbien_LT_PL(df, lst_tencot):
    '''
        return (lst_lientuc,lst_phanloai)
    '''
    col = df.select_dtypes(include='number')
    lst_lientuc = []
    lst_phanloai = []
    for i in lst_tencot:
        if (i in col) and (len(df[i].unique()) > len(df[i])*0.05):
            lst_lientuc.append(i)
        else:
            lst_phanloai.append(i)
    print('Biến liên tục: \n', lst_lientuc)
    print('Biến phân loại: \n',lst_phanloai)
    return (lst_lientuc,lst_phanloai)

In [ ]:
# Phân tích đơn biến: biến phân loại
def phantich_bienphanloai (df, lst_phanloai):
    a = 1
    for i in lst_phanloai:
        print(a,'/',i,'\n')
        table = pd.DataFrame(df[i].value_counts())
        print(table)
        plt.figure(figsize = (5,5))
        sns.barplot(table.index,table[i])
        plt.xticks(rotation = 90)
        plt.show()
        print('-'*20)
        a = a+1

In [ ]:
# Phân tích đơn biến: Biến liên tục:
def phantich_bienlientuc(df, lst_lientuc):
    print(df[lst_lientuc].describe())
    a = 1
    for i in lst_lientuc:
        print(a,'/', i, '/\n')
        i_mode = df[i].mode()[0]
        i_range = df[i].max() - df[i].min()
        print('Phương sai: ',df[i].var())
        print('Độ lệch chuẩn: ', df[i].std())
        print('Độ nghiêng: ', df[i].skew())
        print('Độc dốc: ', df[i].kurt())
        print('Mode: ', i_mode)
        print('Range: ', i_range)
        plt.figure(figsize=(10,5))
        plt.subplot(1,2,1)
        sns.distplot(df[i])
        plt.subplot(1,2,2)
        sns.boxplot(df[i])
        plt.show()
        print('-'*20)
        a = a+1

In [ ]:
# Xử lý ngoại lệ:

def xulyngoaile(df,lst_lientuc):
    a = 1
    for i in lst_lientuc:
        print(a, '/', i, '\n')
        i_mode = df[i].mode()[0]
        i_range = np.ptp(df[i])
        Q1 = np.percentile(df[i],25)
        Q3 = np.percentile(df[i],75)
        i_IQR = Q3 - Q1
        up = Q3 + i_IQR*1.5
        down = Q1 - i_IQR*1.5
        print('Mode: ', i_mode)
        print('Range: ', i_range)
        upoutlier = df.loc[df[i] > up,i]
        downoutlier = df.loc[df[i] < down,i]
        print('outlier trên: \n', upoutlier)
        print('outlier dưới: \n', downoutlier)
        print('-'*20)
        a = a+1

In [ ]:
# Phân tích hai biến Phân loại - Phân loại: sử dụng kiểm định Chi bình phương
def phantich2bien_ploai_ploai(df, lst_phanloai):
    # Giả thiết H0: hai biến độc lập,
    # độ tin cậy 1 - alpha = 0.95
    prob = 0.95
    alpha = 1 - prob
    a = 1
    docLap = []
    phuThuoc = []
    for i in lst_phanloai:
        if lst_phanloai.index(i) == (len(lst_phanloai)-1): break
        else:
            j = lst_phanloai[lst_phanloai.index(i)+1]
            n = lst_phanloai.index(j)
            for j in lst_phanloai[n:]:
                print(a, '/',i,' - ', j)
                table = pd.crosstab(df[i],df[j])
                print(table)
                plt.figure(figsize=(6,4))
                table.plot(kind='bar', stacked = True)
                plt.legend(loc='upper center', bbox_to_anchor=(0.5, 1.5),
                ncol=3, fancybox=True, shadow=True)
                plt.show()            
                print('*** Kiểm định hai biến: ', i,' và ',j,'***')
                print('--- Giả thuyết H0: ', i, ' và ',j, 'độc lập ----')
                stat, p, dof, expected = chi2_contingency(table)
                print(f'p = {p:.10f}')
                print(f'alpha = {alpha:.2f}')
                print('*** KẾT LUẬN ***')
                if p <= alpha:
                    ketLuan = 'KHÔNG ĐỘC LẬP (BÁC BỎ GIẢ THUYẾT H0)'
                    print(f'p <= alpha: Hai biến {i} - {j}: {ketLuan}')
                    phuThuoc.append(i+'-'+j)
                else:
                    ketLuan = 'ĐỘC LẬP (CHẤP NHẬN GIẢ THUYẾT H0)'
                    docLap.append(i+'-'+j)
                    print(f'p > alpha: Hai biến {i} - {j}: {ketLuan}')
                print('-'*20)
                a = a + 1
    return (docLap,phuThuoc)

In [ ]:
# Phân tích hai biến Phân loại - Liên tục
def phantich2bien_ploai_ltuc(df, lst_phanloai, lst_lientuc):
    '''
        return lst_anhhuong
    '''
    a = 1
    # Giả thiết H0: hai biến độc lập,
    # độ tin cậy 1 - alpha = 0.95
    lst_anhhuong = []
    prob = 0.95
    alpha = 1 - prob
    for i in lst_phanloai:
        for n in lst_lientuc:
            print(f'{a}/ {i} - {n}:')
            df_sub = df[[i,n]]
            print(df_sub.head())
            plt.figure(figsize=(12,10))
            sns.boxplot(x = i, y = n, data= df_sub, palette='Set3')
            plt.show()
            str = f'{n} ~ C({i})'
            model = ols(str, data = df_sub).fit()
            anova_table = sm.stats.anova_lm(model, typ = 2)
            print(f'*** Bảng ANOVA ***\n{anova_table}')
            p = anova_table.iat[0,3]
            print(f'alpha = {alpha:.2f}')
            print(f'p = {p:.4f}')
            if p > alpha:
                ketLuan = f'{i} KHÔNG ảnh hưởng đáng kể đển {n}'
                print(ketLuan)
            else:
                ketLuan = f'{i} CÓ ảnh hưởng đáng kể đển {n}'
                anhhuong = f'{i} - {n}'
                lst_anhhuong.append(anhhuong)
                m_comp = pairwise_tukeyhsd (endog=df_sub[n],
                                           groups=df_sub[i],
                                           alpha=alpha)
                print(ketLuan)
                print('*** KIỂM ĐỊNH Tukey ***')
                print(m_comp)
                print('#'*20)
            a = a+1
    return lst_anhhuong

In [ ]:
def phantich2bien_ltuc_ltuc (df, lst_lientuc, n):
    '''
        n: biến liên tục cần dự đoán
        return (lst_tuongquan)
    '''
    print(df[lst_lientuc].corr())
    sns.pairplot(df[lst_lientuc])
    #chọn ra những thuộc tính có tương quan mạnh (hệ số tương quan trong khoảng +- 0.6 đến +-1)để phân tích
    lst_tuongquan=[]
    for i in lst_lientuc:
        a = df[[i,n]].corr().iat[0,1]
        if ((-1<= a <= -0.6) or (0.6 <= a <= 1)):
            lst_tuongquan.append(i)
            print(i,':',a)
    print(lst_tuongquan)
    return(lst_tuongquan)

In [ ]:
def phantich2bien__ltuc_ltuc_chitiet(df,lst_lientuc):
    print(df[lst_lientuc].corr())
    a = 1
    for i in lst_lientuc:
        if lst_lientuc.index(i) == (len(lst_lientuc)-1): break
        else:
            j = lst_lientuc[lst_lientuc.index(i)+1]
            n = lst_lientuc.index(j)
            for j in lst_lientuc[n:]:
                corr = df[[i,j]].corr().iat[0,1]
                if corr == -1:
                    ketLuan = f'Hai biến {i} - {j}: TƯƠNG QUAN ÂM TUYỆT ĐỐI'
                elif (-1 < corr) & (corr <= -0.6 ):
                    ketLuan = f'Hai biến {i} - {j}: TƯƠNG QUAN NGHỊCH BIẾN MẠNH'
                elif (-0.6 < corr) & (corr < -0.3):
                    ketLuan = f'Hai biến {i} - {j}: TƯƠNG QUAN NGHỊCH BIẾN YẾN'
                elif ((-0.3 <= corr) &(corr <= 0.3)):
                    ketLuan = f'Hai biến {i} - {j}: KHÔNG TƯƠNG QUAN'
                elif (0.3 < corr) & (corr < 0.6):
                    ketLuan = f'Hai biến {i} - {j}: TƯƠNG QUAN THUẬN BIẾN YẾU' 
                elif (0.6 <= corr) & (corr < 1):
                    ketLuan = f'Hai biến {i} - {j}: TƯƠNG QUAN THUẬN BIẾN MẠNH' 
                else:
                    ketLuan = f'Hai biến {i} - {j}: TƯƠNG QUAN DƯƠNG TUYỆT ĐỐI'
                print(f'{a}/ {i} - {j}: \nHệ số tương quan: {corr}')
                print(ketLuan)
                plt.figure(figsize=(6,4))
                sns.regplot(df[i],df[j])
                plt.show()
                print('-'*20,'\n')
                a = a+1

In [ ]:
# Chuẩn hóa dữ liệu
def chuanhoa(scale,df,lst_tencot):
    '''
       return df_scale 
    '''
    df_scale =  scale.fit_transform(df)
    df_scale = pd.DataFrame(df_scale,columns=df.columns)
    f, (ax1, ax2) = plt.subplots(ncols=2, nrows=1, figsize=(8,5))
    plt.xlabel('')
    ax1.set_title('Before scaling')
    for i in lst_tencot:
        sns.kdeplot(df[i],ax = ax1, label= i)
    ax1.set_xlabel('')
    ax2.set_title('After scaling')
    for i in lst_tencot:  
        sns.kdeplot(df_scale[i],ax = ax2, label= i)
    # plt.legend(loc='upper center', bbox_to_anchor=(0.0, 1.5),
    #           ncol=3, fancybox=True, shadow=True)
    ax2.set_xlabel('')
    plt.show()
    return df_scale

In [ ]:
# Import Thư viện
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures

from sklearn.pipeline import Pipeline

from sklearn.metrics import mean_squared_error, mean_absolute_error

In [ ]:
def model_dudoan_bienlientuc(train_size, random_state,model,X,y,k,l,m,n):
    '''
        k,l,m,n: định khoảng của mô hình đánh giá biểu đồ yhat_test và y_test
        return(a,b,R_full,R_test,R_train,mse_test,mse_test,mae_test,mae_train,yhat,yhat_test,yhat_train)
    '''
    X_train, X_test, y_train, y_test = train_test_split(X,y, train_size=train_size, random_state=random_state)
    # Train model:
    model.fit(X_train,y_train)
    # dự đoán (kiểm tra) kết quả dựa trên dữ liệu train va test
    yhat_train = model.predict(X_train)
    yhat_test = model.predict(X_test)
    yhat = model.predict(X)
#     a = model.intercept_
#     b = model.coef_
    # tính R^2
    R_full = model.score(X, y)
    R_test = model.score(X_test, y_test)
    R_train = model.score(X_train, y_train)    
    mse_train = mean_squared_error(y_train, yhat_train)
    mse_test = mean_squared_error(y_test, yhat_test)
    mae_train = mean_absolute_error(y_train, yhat_train)
    mae_test = mean_absolute_error(y_test, yhat_test)
    print('Full data R-square:', model.score(X, y))
    print('train data R-square:', model.score(X_train, y_train))
    print('test data R-square:', model.score(X_test, y_test))
    print('-'*20)
    print('train data MSE:', mse_train)
    print('test data MSE:', mse_test)
    print('train data MAE:', mae_train)
    print('test data MAE:', mae_test)
    print('-'*20)
    # Đánh giá mô hình
    plt.figure(figsize=(6,6))
    plt.scatter(yhat_test, y_test)
    plt.xlabel('Model predictions')
    plt.ylabel('True values')
    plt.plot([k,l], [m,n], 'k-', color='r')
    plt.show()
    print('-'*20)
    # Distribution plot
    plt.figure(figsize=(14,6))
    plt.subplot(1,2,1)
    ax1 = sns.distplot(y_train, hist= False, color='r', label= 'Actual Train Values')
    sns.distplot(yhat_train, hist= False, color='b', label= 'Predicted Train Values', ax=ax1)
    plt.legend()
    plt.subplot(1,2,2)
    ax2 = sns.distplot(y_test, hist= False, color='r', label= 'Actual Test Vlaues')
    sns.distplot(yhat_test, hist= False, color='b', label= 'Predicted Train Values', ax=ax2)
    plt.legend()
    plt.show()
    return (model,a,b,R_full,R_test,R_train,mse_test,mse_train,mae_test,mae_train,yhat,yhat_test,yhat_train)

In [ ]:
# Xem xét loại bỏ ngoại lệ
def Xemxetlaoibo_outlier(df,lst_lientuc):
    a = 1
    import scipy
    for i in lst_lientuc:
        print(a,'/', i, '\n')    
        Q1 = np.percentile(df[i],25)
        Q3 = np.percentile(df[i],75)
        i_IQR = scipy.stats.iqr(df[i])
        up = Q3 + i_IQR*1.5
        down = Q1 - i_IQR*1.5
        upoutlier = df.loc[df[i] > up,i]
        downoutlier = df.loc[df[i] < down,i]
        print('outlier trên: ',len(upoutlier),'\n',upoutlier)
        print('outlier dưới: ',len(downoutlier),'\n',downoutlier)
        print('***Xem xét lại điều gì xảy ra với dữ liệu khi có và không có outlier***')
        print('Trươc khi loại outlier: ', df[i].describe())
        print('Sau khi loại outlier: ', df.loc[((down <= df[i])&( df[i]<= up)),i].describe())    
        plt.figure(figsize=(10,5))
        plt.subplot(1,2,1)
        sns.boxplot(df[i])
        plt.title('Dữ liệu gốc')
        plt.subplot(1,2,2)
        sns.boxplot(df.loc[((down <= df[i])&( df[i]<= up)),i])
        plt.title('Loại bỏ outlier')
        plt.show()
        print('-'*20)
        a = a+1

In [ ]:
def model_dudoan_bienphanloai(train_size, random_state,model,X,y):
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, train_size = train_size, random_state= random_state)
    model.fit(X_train,y_train)
    # dự đoán (kiểm tra) kết quả dựa trên dữ liệu train va test
#     a = model.intercept_
#     b = model.coef_
    yhat_train = model.predict(X_train)
    yhat_test = model.predict(X_test)
    yhat = model.predict(X)
    acc_full = accuracy_score(y, yhat)
    acc_test = accuracy_score(y_test, yhat_test)
    acc_train = accuracy_score(y_train, yhat_train)    
    print('acc_full:', acc_full)
    print('acc_test:', acc_test)
    print('acc_train:', acc_train)
    conf_matrix = confusion_matrix(y_true=y_test, y_pred=yhat_test)
    print('Confusion matrix:\n', conf_matrix)
    print(classification_report(y_true=y_test, y_pred=yhat_test))
#     yhat_proba = model.predict_proba(X_test)
#     roc_auc_score(y_test, yhat_proba[:, 1])
#     scores = yhat_proba[:, 1]
#     fpr, tpr, thresholds = roc_curve(y_test, scores)
#     plt.figure(figsize=(6,6))
#     plt.plot([0,1], [0,1], linestyle='--')
#     plt.plot(fpr, tpr, marker='.')
#     plt.xlabel('False Positive Rate')
#     plt.ylabel('True Positive Rate')
#     plt.show()
    return (model,acc_full,acc_test,acc_train,yhat,yhat_test,yhat_train)

In [ ]:
def models_dudoan(test_size_lst, models, X, y, CV, random_state):
    import datetime
    entries = []
    for i in test_size_lst:
        X_train, X_test, y_train, y_test = train_test_split(X,y, test_size= i, random_state=random_state)
        for model in models:
            score_train = []
            score_test = []
            abs_score = []
            time_score = []
            for j in range(CV):
                t1 = datetime.datetime.now()
                model.fit(X_train,y_train)
                t2 = datetime.datetime.now()

                model_name = model.__class__.__name__
                s_train = model.score(X_train,y_train)
                s_test = model.score(X_test,y_test)
                score_train.append(s_train)
                score_test.append(s_test)
                abs_score.append(abs(s_train-s_test))
                time_score.append(round((t2-t1).microseconds/1000,1))
            entries.append([model_name, i, np.array(score_train).mean(),np.array(score_test).mean(), np.array(abs_score).mean(), np.array(time_score).mean()])
        cv_df = pd.DataFrame(entries, columns=['model_name','test_size','score_train_mean','score_test_mean', 'abs_mean','time_score_mean'])
    return cv_df